# Mxnet MNIST mlp

Dimension: 784x10

In [27]:
import mxnet as mx

In [43]:
import numpy as np
from functools import reduce

### Load data

In [28]:
from mnist import MNIST
#you can find python-mnist source code on https://github.com/sorki/python-mnist

datahandler = MNIST('../mnist_data_loader') #change for data path
train_data = datahandler.load_training()

In [29]:
image_array = np.asarray(train_data[0])

In [30]:
label_list = []
for i in train_data[1]:
    l = [0]*10
    l[i] = 1
    label_list.append(l)

label_array = np.asarray(label_list)

In [51]:
#set data slices
train_finish = 50000
eval_finish = 55000
predict_start = 55000

### Create iterators for training, evaluating and testing

In [32]:
train_iter = mx.io.NDArrayIter(
    
    data = image_array[:train_finish],
    label = label_array[:train_finish],
    batch_size = 100,

)

In [33]:
val_iter = mx.io.NDArrayIter(
    
    data = image_array[train_finish:eval_finish],
    label = label_array[train_finish:eval_finish],
    batch_size = 100,

)

In [57]:
test_iter = mx.io.NDArrayIter(
    
    data = image_array[predict_start:],
    batch_size = 1,

)

### Create and train model

In [34]:
data = mx.symbol.Variable('data')
fc1  = mx.symbol.FullyConnected(data = data, num_hidden=784)
act1 = mx.symbol.Activation(data = fc1, act_type="sigmoid")
fc2  = mx.symbol.FullyConnected(data = act1, num_hidden=10)
mlp  = mx.symbol.SoftmaxOutput(data = fc2, name = 'softmax')

In [35]:
model = mx.model.FeedForward(
    
    symbol = mlp,
    num_epoch = 10,
    learning_rate = .01

)

In [ ]:
model.fit(X = train_iter, eval_data = val_iter)

### Make predictions

In [121]:
p = model.predict(X = test_iter)

In [122]:
len(p) #check number of predictions

5000

# Metrics

### Confusion matrix
Useful for metric calculation

In [123]:
conf_mtx = np.zeros([10, 10])
for j in range(0, len(p)):
    prediction = p[j]
    max_pred = reduce(lambda x, y: (x if prediction[x] > prediction[y] else y), range(0,10))
    actual_label = list(filter(lambda i: label_array[predict_start + j][i], range(0,10)))[0]
    conf_mtx[max_pred][actual_label] += 1
print(conf_mtx)
    

[[ 477.    0.    2.    0.    1.    2.    0.    0.    1.    0.]
 [   0.  526.    0.    1.    4.    0.    0.    2.    3.    1.]
 [   2.    1.  480.    2.    0.    2.    0.    1.    2.    1.]
 [   2.    0.    2.  498.    0.    8.    0.    2.    6.    4.]
 [   1.    0.    2.    0.  486.    1.    0.    3.    0.    5.]
 [   0.    0.    1.    4.    0.  434.    1.    0.    6.    2.]
 [   2.    1.    1.    0.    1.    8.  479.    0.    0.    0.]
 [   2.    0.    2.    0.    1.    1.    0.  553.    1.    2.]
 [   1.    2.    3.    2.    0.    1.    2.    1.  472.    1.]
 [   2.    0.    0.    2.    6.    1.    0.    1.    3.  467.]]


### Accuracy

In [124]:
acc = (sum(conf_mtx[i][i] for i in range(0, 10))/5000)

In [125]:
print('Accuracy: {:.5f}'.format(acc))

Accuracy: 0.97440


### Recall

In [126]:
recall = lambda i: (conf_mtx[i][i]/sum(conf_mtx[j][i] for j in range(0,10)))

In [127]:
recall_sum = 0
for i in range(0,10):
    rcl = recall(i)
    recall_sum += rcl
    print('Recall {}: {:.5f}'.format(i, rcl))
print()
print('Recall mean: {:.5f}'.format(recall_sum/10))

Recall 0: 0.97546
Recall 1: 0.99245
Recall 2: 0.97363
Recall 3: 0.97839
Recall 4: 0.97395
Recall 5: 0.94760
Recall 6: 0.99378
Recall 7: 0.98224
Recall 8: 0.95547
Recall 9: 0.96687

Recall mean: 0.97398


### Precision

In [128]:
precision = lambda i: (conf_mtx[i][i]/sum(conf_mtx[i][j] for j in range(0,10)))

In [129]:
precision_sum = 0
for i in range(0,10):
    label_precision = precision(i)
    precision_sum += label_precision
    print('Precision {}: {:.5f}'.format(i, label_precision))
print()
print('Precision mean: {:.5f}'.format(precision_sum/10))

Precision 0: 0.98758
Precision 1: 0.97952
Precision 2: 0.97760
Precision 3: 0.95402
Precision 4: 0.97590
Precision 5: 0.96875
Precision 6: 0.97358
Precision 7: 0.98399
Precision 8: 0.97320
Precision 9: 0.96888

Precision mean: 0.97430
